In [48]:
import pandas as pd
import numpy as np
import duckdb
from pathlib import Path
import os

LOADING DATA - *Lending Club dataset 2018-2019*

In [49]:
os.chdir('c:\\Renzo\\Projects\\credit-risk-ai\\')
os.getcwd()

'c:\\Renzo\\Projects\\credit-risk-ai'

In [50]:
# As this is the exploration phase, we only need to check a sample of the data, so lets pick 1000

con = duckdb.connect()

df = con.execute("""
    SELECT *
    FROM read_csv_auto(
        'data/accepted_2007_to_2018Q4.csv',
        types={'id': 'VARCHAR'}  -- Fix the footer issue
    )
    WHERE issue_d LIKE '%-2017' OR issue_d LIKE '%-2018'
    LIMIT 1000
""").df()

print(df.shape)
df.head()

(1000, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,130954621,None,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
1,130964697,None,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
2,130955326,None,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
3,130504052,None,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
4,130956066,None,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN


We are building a supervised learning model. Hence, we need a target. We will focus on loans that we have a real outcome for (i.e. loans that we know whether the borrower payed the loan back or defaulted.) This, we will create a binary target variable to use as a ground truth. In this case, loans with status labeled as **charged off** or **default** will be = 1 (flagged), and the rest will be 0.

In [51]:
df_filtered = con.execute("""
    SELECT 
        CASE WHEN loan_status IN ('Default', 'Charged Off') THEN 1 ELSE 0 END AS target,
        loan_amnt, int_rate, installment, annual_inc, dti, revol_bal, revol_util,
        total_acc, open_acc, pub_rec, fico_range_low, fico_range_high,
        inq_last_6mths, delinq_2yrs, earliest_cr_line, term,
        home_ownership, purpose, grade, sub_grade, emp_length,
        verification_status, addr_state, issue_d
    FROM read_csv_auto(
        'data/accepted_2007_to_2018Q4.csv',
        types={'id': 'VARCHAR'}
    )
    WHERE 
        loan_status IN ('Fully Paid', 'Charged Off', 'Default')
        AND (issue_d LIKE '%-2017' OR issue_d LIKE '%-2018')
        AND annual_inc > 0
        AND dti IS NOT NULL
        AND revol_util IS NOT NULL
""").df()

print(f'Shape of full filtered dataset {df_filtered.shape}')
print(f'% of defaulted or charged off loans: {df_filtered['target'].mean():.2%}')
df.head()


Shape of full filtered dataset (225081, 25)
% of defaulted or charged off loans: 21.30%


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,130954621,None,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
1,130964697,None,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
2,130955326,None,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
3,130504052,None,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
4,130956066,None,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN


In [52]:
print(df_filtered['issue_d'].str[-4:].value_counts())

issue_d
2017    168988
2018     56093
Name: count, dtype: int64


Now that we have a ground truth, we will turn the rest of the data (columns) into features that the model can learn from, and drop the rest.

In [53]:
df = df_filtered.copy()

# Creating column: credit_age

df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')

df['credit_age'] = df['issue_d'] - df['earliest_cr_line']
df['credit_age'] = df['credit_age'].dt.days/365.25
df['credit_age'] = df['credit_age'].clip(lower=0)

# Creating column: payment_to_income

df['payment_to_income'] = (df['installment']*12)/df['annual_inc'].replace(0, np.nan)

# Creating column: revol_utilization_trend

df['revol_utilization_trend'] = df['revol_bal']/(df['annual_inc']+1)

# Creating column: log_annual_inc, log_loan_amnt, log_revol_bal

df['log_annual_inc'] = np.log1p(df['annual_inc'])
df['log_loan_amnt'] = np.log1p(df['loan_amnt'])
df['log_revol_bal'] = np.log1p(df['revol_bal'])

In [54]:
df['term'].value_counts()

term
36 months    169443
60 months     55638
Name: count, dtype: int64

In [55]:

# Creating column: term_months

df['term_months'] = df['term'].map({'36 months': 36, '60 months': 60})

# Creating column: fisco_avg

df['fico_avg'] = (df['fico_range_low'] + df['fico_range_high'])/2


In [56]:
cols_to_drop = ['earliest_cr_line', 'term', 'fico_range_low', 'fico_range_high']
df_clean = df.drop(columns=cols_to_drop)

In [57]:
print(f"New shape: {df_clean.shape}")
print(f"New columns: {len(list(df_clean.columns))}")
df_clean[['target', 'credit_age', 'payment_to_income', 'fico_avg', 'term_months']].head()

New shape: (225081, 29)
New columns: 29


,target,credit_age,payment_to_income,fico_avg,term_months
0,0,20.161533,0.021485,762.0,NaN
1,0,16.580424,0.036225,677.0,NaN
2,0,11.000684,0.069732,697.0,NaN
3,0,17.913758,0.240714,682.0,NaN
4,0,9.327858,0.079192,667.0,NaN


In [58]:
# Save clean

df_clean.to_parquet('data/processed.parquet', index=False)